In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%run autoencoder.py 64 2 0.25 --uneven --model_type gru --sigma 1e-9 --sim_type autoencoder/uneven/noise0

Using TensorFlow backend.


[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/autoencoder/uneven/noise0/gru_064_x2_2m03_drop25
Loading /Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/autoencoder/uneven/noise0/gru_064_x2_2m03_drop25/weights.h5...


In [10]:
model.layers

In [6]:
%%time
encode_layer = np.where(['RepeatVector' in str(l) for l in model.layers])[0].item() - 1
if args.even:
    encode = K.function([model.layers[0].input], [model.layers[encode_layer].output])
    encoding, = encode([X[:]])
else:
    encode = K.function([model.layers[0].input, model.layers[args.num_layers + 1].input],
                        [model.layers[encode_layer].output])
    encoding, = encode([X[:], X[:, :, 0:1]])

ValueError: Cannot feed value of shape (51000, 250, 1) for Tensor 'cond/Merge:0', which has shape '(?, ?, 64)'

In [ ]:
i = -1

In [ ]:
i += 1
if args.even:
    t = np.linspace(0, 2 * np.pi, X.shape[1])
    m = X[i, :, 0]
    pred_i = model.predict(X[[i]])[0, :, 0]
else:
    inds = (X[i, :, -1] >= 0.)
    t = X[i, inds, 0]
    m = X[i, inds, 1]
    e = X[i, inds, 2]
    #plt.plot(t, m, 'o')
    plt.errorbar(t, m, e, None, 'o')
    pred_i = model.predict({'main_input': X[i:i+1], 'aux_input': X[i:i+1, :, [0, 2]]})[0, inds, 0]

plt.plot(t, m, 'o')
plt.plot(t, pred_i, 'o')

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

#plt.legend(['Noisy', 'Original'])

In [ ]:
np.mean((m - pred_i) ** 2)

In [ ]:
S = np.corrcoef(encoding, Y[:encoding.shape[0]], rowvar=0)[:8, 8:]
print(pd.DataFrame(S))
plt.imshow(S, cmap='viridis', interpolation='none')
plt.colorbar()

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 0], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 1], Y[:encoding.shape[0], 1], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 2], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 3], Y[:encoding.shape[0], 3], kind='hex')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

model = RandomForestRegressor(n_estimators=128)
#model = ElasticNet()
#model = SVR(kernel='rbf')

model.fit(encoding, Y[:encoding.shape[0], 0])

In [ ]:
encoding_test = encode([X[test]])[0]
Y_pred = model.predict(encoding_test)

In [ ]:
#sns.jointplot(Y[test, 0], Y_pred[:, 0])
sns.jointplot(Y[test, 0], Y_pred)